In [1]:
%%time
import manager_dataset as MD
import question_processing as QP
import information_retrieval as IR
from gensim.models import KeyedVectors

Wall time: 4.26 s


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Settings

Questions recive all the questions treated

In [2]:
questions = MD.questions(treated=True)
questions = QP.remove_incosiderate_classes(questions)  # Question with incosiderate class is removed

The questions that has an answer will be used for test while another questions will be used for train.

In [3]:
train_questions, test_questions = MD.split_questions(questions)

## Question Processing

### Question Classification

Questions to train and test

In [4]:
qc_X_train, qc_y_train = QP.separete_questions(train_questions)

Load Word2Vec model

In [5]:
%%time
w2v_patch = 'data/word_embedding/cbow_s300.txt'
w2v_patch = 'data/word_embedding/cbow_s50.txt'  # Fast test
w2v = KeyedVectors.load_word2vec_format(w2v_patch, unicode_errors="ignore")
model_w2v = {w: vec for w, vec in zip(w2v.index2word, w2v.syn0)}

Wall time: 38.7 s


Training

In [6]:
%%time
classifier = QP.svm_classifier()
vectorizer = QP.HybridVectorizer(model_w2v)
#vectorizer = QP.MeanEmbeddingVectorizer(model_w2v)
#vectorizer = QP.TfidfHybridVectorizer(model_w2v)
#vectorizer = QP.SequenceHybridVectorizer(model_w2v)
#vectorizer = QP.count_vectorizer(model_w2v)
#vectorizer = QP.tfidf_vectorizer(model_w2v)
model = QP.train_model(qc_X_train, qc_y_train, classifier, vectorizer)

Wall time: 411 ms


Answer type is predicted for all questions

In [7]:
questions = QP.predict_answer_type(model, test_questions)

#### Testing

In [8]:
qc_X_test, qc_y_test = QP.separete_questions(test_questions)
print('Size train:',len(qc_y_train),'  Size test:',len(qc_y_test))

Size train: 2147   Size test: 826


In [9]:
%matplotlib notebook
confusion_matrix = QP.testing(model, qc_X_test, qc_y_test)
QP.plot_confusion_matrix(confusion_matrix, ['DEFINITION', 'LOCATION', 'MEASURE', 'ORGANIZATION', 'PERSON', 'TIME'], normalize=True)

Accuracy: 0.889830508475
F1 Score: 0.880864987979


<IPython.core.display.Javascript object>

### Query Formulation

All test questions will recive the query for IR module. 

In [4]:
test_questions = QP.queryFormulation(test_questions)

## Information Retrieval

### Document Retrieval

In [5]:
import information_retrieval as IR
ir = IR.InformationRetrieval()

Start the IR module. If the documents was not indexed then they will be.
When reset is True, all indexed documents is removed and indexed again.

In [7]:
ir.start(reset=True)

Indexing:
File: 51511 lines: 
51511/999
51511/1999
51511/2999
51511/3999
51511/4999
51511/5999
51511/6999
51511/7999
51511/8999
51511/9999
51511/10999
51511/11999
51511/12999
51511/13999
51511/14999
51511/15999
51511/16999
51511/17999
51511/18999
51511/19999
51511/20999
51511/21999
51511/22999
51511/23999
51511/24999
51511/25999
51511/26999
51511/27999
51511/28999
51511/29999
51511/30999
51511/31999
51511/32999
51511/33999
51511/34999
51511/35999
51511/36999
51511/37999
51511/38999
51511/39999
51511/40999
51511/41999
51511/42999
51511/43999
51511/44999
51511/45999
51511/46999
51511/47999
51511/48999
51511/49999
51511/50999
51511/51510
File: 51674 lines: 
51674/999
51674/1999
51674/2999
51674/3999
51674/4999
51674/5999
51674/6999
51674/7999
51674/8999
51674/9999
51674/10999
51674/11999
51674/12999
51674/13999
51674/14999
51674/15999
51674/16999
51674/17999
51674/18999
51674/19999
51674/20999
51674/21999
51674/22999
51674/23999
51674/24999
51674/25999
51674/26999
51674/27999
51674/28999


In [8]:
test_questions = ir.retrievalDocuments(test_questions)

#### Testing

In [25]:
import numpy as np

precisions = []
recalls = []
f_scores = []
b = 0.5
for question in test_questions:
    relevants = []
    for answer in question['answers']:
        if answer['doc'] is not None and len(answer['doc']) > 0:
            relevants.append(answer['doc'])
    total_relevants = len(relevants)
    if total_relevants == 0:
        continue
    relevents_retrieval = 0
    for retrieval in question['retrieval']:
        if retrieval in relevants:
            relevents_retrieval += 1
    if len(question['retrieval']) == 0:
           # print(question['query'])
            precision = 0
    else:
        precision = relevents_retrieval / len(question['retrieval'])
    precisions.append(precision)
    recall = relevents_retrieval / total_relevants
    recalls.append(recall)
    if (precision + recall) == 0:
        f_score = 0
    else:
        f_score = 2 * ((precision * recall) / (precision + recall))
    f_scores.append(f_score)

print('Precision: '+str(np.mean(precisions)))
print('Recall: '+str(np.mean(recalls)))
print('F-Score: '+str(np.mean(f_scores)))

Precision: 0.0274725274725
Recall: 0.201176072605
F-Score: 0.0459659817168


In [16]:
test_questions[0]

{'answers': [{'answer': '\nMilão\n', 'doc': 'PUBLICO-19940724-039', 'n': '2'}],
 'category': 'F',
 'class': 'LOCATION',
 'extracts': [],
 'id': 2,
 'id_org': '0453',
 'ling': 'IT',
 'predic_class': '',
 'query': 'text:Em text:que text:cidade text:se text:encontra text:a text:prisão text:de text:San text:Vittore?',
 'question': 'Em que cidade se encontra a prisão de San Vittore?',
 'restriction': '',
 'retrieval': ['FSP951112-048',
  'FSP950411-054',
  'PUBLICO-19940720-044',
  'FSP950903-055',
  'FSP940120-172',
  'FSP951130-150',
  'FSP951011-071',
  'PUBLICO-19940116-068',
  'FSP950825-051',
  'FSP940120-174'],
 'type': 'LOCATION',
 'year': '2004'}

In [3]:
ir.stop()